# Forecasting crime with Facebook Prophet

#### 1. Import libraries and data

In [294]:
import pandas as pd 
import numpy as np 


from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Ridge

from prophet import Prophet 
from prophet.plot import plot_plotly, plot_components_plotly
import matplotlib.pyplot as plt 
%matplotlib inline

import itertools

import warnings
warnings.simplefilter(action='ignore', category= FutureWarning)

import holidays

import plotly.graph_objs as go

from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_yearly, add_changepoints_to_plot
plt.style.use('fivethirtyeight')

In [295]:
# Read the data
df = pd.read_csv('../data/lrpd-clean.csv')


#### 2. Reducing memory helps us do feature engineering faster and more efficiently

In [296]:
# Reduce memory like we did in data cleanup
def reduce_mem_usage(df, category=False):
  start_mem = df.memory_usage().sum() / 1024 **2
  print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

  for col in df.columns:
    col_type = df[col].dtype

    if col_type != object:
      c_min = df[col].min()
      c_max = df[col].max()
      if str(col_type)[:3] == 'int':
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
          df[col] = df[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
          df[col] = df[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
          df[col] = df[col].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
          df[col] = df[col].astype(np.int64)
      else:
        if c_min  > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
          df[col] = df[col].astype(np.float16)
        elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
          df[col] = df[col].astype(np.float32)
        else:
          df[col] = df[col].astype(np.float64)
    else:
      if category:
        df[col] = df[col].astype('category')

  end_mem = df.memory_usage().sum() / 1024 ** 2
  print('Memory usage after optimizations: {:.2f} MB'.format(end_mem))
  print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
  return df

df = reduce_mem_usage(df)

Memory usage of dataframe is 10.60 MB
Memory usage after optimizations: 5.47 MB
Decreased by 48.4%


#### 3. Set correct data types

In [297]:
df["INCIDENT_DATE"] = pd.to_datetime(df["INCIDENT_DATE"])
df.dtypes

INCIDENT_NUMBER                object
INCIDENT_DATE          datetime64[ns]
LOCATION_DISTRICT             float16
OFFENSE_DESCRIPTION            object
WEAPON_TYPE                    object
ZIP                           float32
LATITUDE                      float16
LONGITUDE                     float16
WEEK_OF_MONTH                    int8
YEAR                            int16
DAY                              int8
DAY_OF_YEAR                     int16
MONTH                            int8
CRIME_TYPE                     object
RISK_TYPE                      object
RISK_TYPE_BC                     int8
dtype: object

#### 4. Prophet takes a dataframe with two columns: ds and y.
* The ds (datestamp) column should be of a format expected by Pandas, ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp.
*  The y column must be numeric, and represents the measurement we wish to forecast.

In [298]:
df = df.groupby(pd.Grouper(key='INCIDENT_DATE', freq='D')).size().reset_index(name='INCIDENT_COUNT')
df.columns = ['ds', 'y']

In [299]:
len(df)

2242

#### 5. Make a baseline model using Prophet to see how well the model does without adding regressors, holidays or hyperparameter tuning.

In [300]:
m = Prophet()
m.fit(df)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)

df_cv = cross_validation(m, initial = '2000 days', period = '60 days', horizon='7 days')
df_p_baseline = performance_metrics(df_cv)
df_p_baseline.head()


14:59:10 - cmdstanpy - INFO - Chain [1] start processing
14:59:11 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 0/4 [00:00<?, ?it/s]14:59:11 - cmdstanpy - INFO - Chain [1] start processing
14:59:11 - cmdstanpy - INFO - Chain [1] done processing
 25%|██▌       | 1/4 [00:00<00:00,  4.93it/s]14:59:11 - cmdstanpy - INFO - Chain [1] start processing
14:59:11 - cmdstanpy - INFO - Chain [1] done processing
 50%|█████     | 2/4 [00:00<00:00,  5.67it/s]14:59:11 - cmdstanpy - INFO - Chain [1] start processing
14:59:11 - cmdstanpy - INFO - Chain [1] done processing
 75%|███████▌  | 3/4 [00:00<00:00,  5.60it/s]14:59:12 - cmdstanpy - INFO - Chain [1] start processing
14:59:12 - cmdstanpy - INFO - Chain [1] done processing
100%|██████████| 4/4 [00:00<00:00,  4.83it/s]


,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,1 days,107.098094,10.348821,9.878269,0.307336,0.243401,0.266595,0.75
1,2 days,29.270193,5.410193,4.610489,0.133948,0.096332,0.122737,0.75
2,3 days,40.349251,6.352106,5.148252,0.138773,0.136381,0.136281,1.00
3,4 days,21.902924,4.680056,3.377001,0.112654,0.068754,0.100913,1.00
4,5 days,74.279860,8.618576,8.002118,0.250732,0.250488,0.221255,0.75


#### 6. Lets add holidays to the model to see how well it does.

In [301]:
# After zooming into the data, we see that arround christmas and 4th of july, we have an affect because of the holidays.
christmas = pd.DataFrame({'holiday': 'Christmas',
  'ds': pd.to_datetime(['2017-12-24','2018-12-24','2019-12-24','2020-12-24', '2021-12-24', '2022-12-24']),
  'lower_window': -2,
  'upper_window': 5,
})

fourth_of_july = pd.DataFrame({'holiday': 'Fourth of July',
    'ds': pd.to_datetime(['2017-07-04','2018-07-04','2019-07-04','2020-07-04', '2021-07-04', '2022-07-04']),
    'lower_window': -1,
    'upper_window': 1,
})

m = Prophet(seasonality_mode='multiplicative', weekly_seasonality=True, daily_seasonality=True, holidays=christmas)
m.add_country_holidays(country_name='US')
m.fit(df)
future = m.make_future_dataframe(periods= 365)
forecast = m.predict(future)

fig = go.Figure()

fig.add_trace(go.Scatter(x=df['ds'], y=df['y'], name='Actual',))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicted',))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['holidays'].values, name='Holidays',))
fig.show()

14:59:12 - cmdstanpy - INFO - Chain [1] start processing
14:59:12 - cmdstanpy - INFO - Chain [1] done processing


Cross validate the model in order to see how well it does across many folds of the data.

In [302]:
df_cv = cross_validation(m, initial = '2000 days', period = '30 days', horizon='7 days')

  0%|          | 0/8 [00:00<?, ?it/s]14:59:13 - cmdstanpy - INFO - Chain [1] start processing
14:59:13 - cmdstanpy - INFO - Chain [1] done processing
 12%|█▎        | 1/8 [00:00<00:03,  1.80it/s]14:59:14 - cmdstanpy - INFO - Chain [1] start processing
14:59:14 - cmdstanpy - INFO - Chain [1] done processing
 25%|██▌       | 2/8 [00:01<00:03,  1.58it/s]14:59:14 - cmdstanpy - INFO - Chain [1] start processing
14:59:15 - cmdstanpy - INFO - Chain [1] done processing
 38%|███▊      | 3/8 [00:01<00:02,  1.79it/s]14:59:15 - cmdstanpy - INFO - Chain [1] start processing
14:59:16 - cmdstanpy - INFO - Chain [1] done processing
 50%|█████     | 4/8 [00:02<00:02,  1.40it/s]14:59:16 - cmdstanpy - INFO - Chain [1] start processing
14:59:16 - cmdstanpy - INFO - Chain [1] done processing
 62%|██████▎   | 5/8 [00:03<00:02,  1.47it/s]14:59:16 - cmdstanpy - INFO - Chain [1] start processing
14:59:17 - cmdstanpy - INFO - Chain [1] done processing
 75%|███████▌  | 6/8 [00:04<00:01,  1.36it/s]14:59:17 - cmds

This is what cross validation looks like. Yhat represents the predicted target variable and y represents the actual target variable.

In [303]:
df_cv.head()

,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,2022-07-19,41.330902,31.677353,50.984214,36,2022-07-18
1,2022-07-20,41.546306,31.701852,51.529759,59,2022-07-18
2,2022-07-21,42.034829,32.222437,52.057712,40,2022-07-18
3,2022-07-22,43.967338,34.009859,53.155976,47,2022-07-18
4,2022-07-23,39.392435,30.449101,49.374480,38,2022-07-18


Evaluation metrics for each of the cutoff's in the cross validation

In [304]:
df_p = performance_metrics(df_cv)
df_p.head(5)

,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,1 days,69.495232,8.336380,7.405780,0.227127,0.221544,0.203383,0.750
1,2 days,48.639261,6.974185,4.626244,0.111059,0.090533,0.112995,0.875
2,3 days,47.162150,6.867470,5.437873,0.129284,0.128913,0.137916,0.875
3,4 days,30.873352,5.556379,4.378553,0.113867,0.062709,0.116240,0.875
4,5 days,62.320300,7.894321,6.760927,0.243932,0.153487,0.205274,0.750


## Putting it all together

* Now we can start putting everything together by hyperparameter tuning, adding holidays, and eventually adding weather data as well.

In [305]:
df = pd.read_csv('lrpd-clean.csv')
df['INCIDENT_DATE'] = pd.to_datetime(df['INCIDENT_DATE'])
df = df.groupby(pd.Grouper(key='INCIDENT_DATE', freq='D')).size().reset_index(name='INCIDENT_COUNT')
df.columns = ['ds', 'y']
df.shape

(2242, 2)

In [306]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.1],
    'seasonality_prior_scale': [0.01, 1.0],
    'holidays_prior_scale': [0.01, 0.1],
    'seasonality_mode': ['additive', 'multiplicative'],
    'daily_seasonality': [True, False],
    'weekly_seasonality': [True, False],
    'yearly_seasonality': [True, False],
    'holidays': [christmas, fourth_of_july],
}

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []

len(all_params)

256

I commented out this cell to save time on finding the best parameters as it would take a long time to run.

If you would like to run it, uncomment it and run it. 

Make sure to comment the next cell.

In [307]:
# for params in all_params:
#     m = Prophet(**params).fit(df)
#     df_cv = cross_validation(m, initial = '1940 days', period = '30 days', horizon='30 days', parallel="processes")
#     df_p = performance_metrics(df_cv, rolling_window=1)
#     rmses.append(df_p['rmse'].values[0])

# # Find the best parameters
# tuning_results = pd.DataFrame(all_params)
# tuning_results['rmse'] = rmses
# print(tuning_results)

# # Python
# best_params = all_params[np.argmin(rmses)]
# print(best_params)

In [308]:
best_params = {'changepoint_prior_scale': 0.1,
 'seasonality_prior_scale': 1.0,
 'holidays_prior_scale': 0.1,
 'seasonality_mode': 'multiplicative',
 'daily_seasonality': True,
 'weekly_seasonality': True,
 'yearly_seasonality': False,
 'holidays': christmas,
 }

#### We fit the model with the best parameters, and the holidays data.

In [309]:
m = Prophet(**best_params)
m.add_country_holidays(country_name='US')
m.fit(df)
future = m.make_future_dataframe(periods= 365)
forecast = m.predict(future)
df_cv = cross_validation(m, initial = '2000 days', period = '60 days', horizon='7 days', parallel="processes")
df_p = performance_metrics(df_cv, rolling_window=1)


14:59:18 - cmdstanpy - INFO - Chain [1] start processing
14:59:20 - cmdstanpy - INFO - Chain [1] done processing
14:59:21 - cmdstanpy - INFO - Chain [1] start processing
14:59:21 - cmdstanpy - INFO - Chain [1] start processing
14:59:21 - cmdstanpy - INFO - Chain [1] start processing
14:59:21 - cmdstanpy - INFO - Chain [1] start processing
14:59:22 - cmdstanpy - INFO - Chain [1] done processing
14:59:22 - cmdstanpy - INFO - Chain [1] done processing
14:59:22 - cmdstanpy - INFO - Chain [1] done processing
14:59:22 - cmdstanpy - INFO - Chain [1] done processing


#### We add weather data from the local Little Rock airport weather station to the model

It is important to note that we will also need the weather data for the future in order to make forecasts on the future. 

We can obtain that data by using a weather data API to get the weather forecast for the horizon.

In [310]:
weather = pd.read_csv('weather.csv')
best_data = weather[weather['NAME'] == 'LITTLE ROCK AIRPORT ADAMS FIELD, AR US']
columns_to_keep = ['DATE', 'AWND', 'PRCP', 'SNWD', 'SNOW', 'TMAX', 'TMIN']
best_data = best_data[columns_to_keep]
best_data.fillna(0, inplace=True)
best_data = best_data[best_data['DATE'] < '2023-02-21']
best_data['DATE'] = pd.to_datetime(best_data['DATE'])
best_df = pd.merge(df, best_data, left_on='ds', right_on='DATE', how='inner')
best_df.drop(columns=['DATE'], inplace=True)

In [311]:
best_df.head()

,ds,y,AWND,PRCP,SNWD,SNOW,TMAX,TMIN
0,2017-01-01,54,6.93,0.01,0.0,0.0,46.0,43.0
1,2017-01-02,45,5.37,0.20,0.0,0.0,56.0,45.0
2,2017-01-03,49,7.61,0.00,0.0,0.0,57.0,40.0
3,2017-01-04,48,9.62,0.00,0.0,0.0,41.0,26.0
4,2017-01-05,39,8.50,0.00,0.0,0.0,37.0,29.0


Add the regressors to the model and train it again.

In [312]:
finalDf = best_df.copy()
list2 = ['AWND', 'PRCP', 'SNWD', 'SNOW', 'TMAX', 'TMIN']
estimator = Ridge()
selector =  RFECV(estimator, step=10, cv=10)
selector = selector.fit(finalDf[list2], finalDf['y'])
# 
to_keep = finalDf[list2].columns[selector.support_]


m = Prophet(**best_params)
m.add_country_holidays(country_name='US')

# add regressors to the dataframe
for f in to_keep:    
    df[f] = finalDf[f]
    m.add_regressor(f)

m.fit(df)

14:59:23 - cmdstanpy - INFO - Chain [1] start processing
14:59:24 - cmdstanpy - INFO - Chain [1] done processing


In [313]:
df_cv = cross_validation(m, initial = '2000 days', period = '60 days', horizon='7 days')
df_p2 = performance_metrics(df_cv, rolling_window=1)

  0%|          | 0/4 [00:00<?, ?it/s]14:59:24 - cmdstanpy - INFO - Chain [1] start processing
14:59:24 - cmdstanpy - INFO - Chain [1] done processing
 25%|██▌       | 1/4 [00:00<00:01,  1.65it/s]14:59:24 - cmdstanpy - INFO - Chain [1] start processing
14:59:25 - cmdstanpy - INFO - Chain [1] done processing
 50%|█████     | 2/4 [00:01<00:01,  1.37it/s]14:59:25 - cmdstanpy - INFO - Chain [1] start processing
14:59:26 - cmdstanpy - INFO - Chain [1] done processing
 75%|███████▌  | 3/4 [00:02<00:00,  1.26it/s]14:59:26 - cmdstanpy - INFO - Chain [1] start processing
14:59:27 - cmdstanpy - INFO - Chain [1] done processing
100%|██████████| 4/4 [00:03<00:00,  1.32it/s]


#### Now we compare the metrics for all 3 stages of model fitting and see how they compare. 
As we can see, the model with the weather data performs the best.

In [314]:
comparison = pd.DataFrame()
comparison['baseline'] = df_p_baseline.mean(axis = 0)[1:]
comparison['raw'] = df_p.mean(axis = 0)[1:]
comparison['covariates'] = df_p2.mean(axis = 0)[1:]
print(comparison)

           baseline       raw covariates
mse       52.607482   46.1215  43.925174
rmse       6.881179  6.791281   6.627607
mae         5.89508  5.904483    5.64883
mape          0.183  0.176042   0.165862
mdape      0.147671  0.136131   0.161533
smape      0.164388  0.165053   0.158754
coverage   0.857143  0.857143   0.892857
